In [ ]:
!pip install deap
!pip install scoop
!pip install multiprocessing
!pip install joblib
import multiprocessing
import itertools as it
import concurrent.futures
import time
import numpy as np
import matplotlib.pyplot as plt
import typing
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_probability as tfp
import multiprocessing
import deap
from deap import algorithms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 160 kB 25.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 603 kB 12.4 MB/s 
  Created wheel for scoop: filename=scoop-0.7.1.1-py3-none-any.whl size=72141 sha256=9390e1421d2b2f7202ed1bd871853bad42493a0408bd531947b6e05e5c67daa1
  Stored in directory: /root/.cache/pip/wheels/24/19/e9/6e3e7c0323cc36bf1e4993d69b2db27d6b4e806ed57d411f44
Successfully built scoop


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 108 kB 19.7 MB/s 
     |████████████████████████████████| 129 kB 35.1 MB/s 
     |████████████████████████████████| 106 kB 16.8 MB/s 
     |████████████████████████████████| 103 kB 35.6 MB/s 
ERROR: Could not find a version that satisfies the requirement multiprocessing (from versions: 2.6.0-0.1, 2.6.0.2, 2.6.1.1, 2.6.2.1)
ERROR: No matching distribution found for multiprocessing
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class DO:

  def __init__(self,dim: int,DO_init):
    self.dim=dim
    self.M=DO_init
    #print("The input DO is: \n",self.M.numpy())
    #print(self.test())

  def eig(self,vect=False):
    ''' returns eigen values and eigen vectors of the DO.'''

    #print('**********************')
    #print('Eig-Vals using numpy:')
    #eval_,evec_=np.linalg.eigh(self.M.numpy())
    #print(eval_)

    eval,evec=tf.linalg.eigh(self.M)

    #print('Eig-Vals using tf:')
    #print(eval)
    #print('**********************')

    if vect:
      return (eval,evec)
    else:
      return eval


  def test(self):
    ''' Checks if the DO is a valid Density operator.'''
    TOL=1e-6
    if np.allclose(np.trace(self.M.numpy()),1) and np.trace(np.linalg.matrix_power(self.M.numpy(),2))<=1+TOL and \
       np.all(self.eig().numpy()>=0-TOL) and np.array_equal(np.shape(self.M.numpy()),np.array([self.dim,self.dim])):
       None
    else:

     if ~np.allclose(np.trace(self.M.numpy()),1):
       print('Wrong Trace:', end=' '); print(np.trace(self.M.numpy()))

     if ~(np.trace(np.linalg.matrix_power(self.M.numpy(),2))<=1+TOL):
       print('Wrong Squared Trace:',end=' '); print(np.trace(np.linalg.matrix_power(self.M.numpy(),2)))

     if ~(np.all(self.eig().numpy()>=0-TOL)):
       print('Not positive:');print(self.eig().numpy())

     if ~(np.array_equal(np.shape(self.M.numpy()),np.array([self.dim,self.dim]))):
       print('Size Problem')
       print('DO dim:',end=' ');print(np.shape(self.M.numpy()))
       print('Given dim:',end= ' ');print(self.dim)

     raise Exception("Invalid Density Operator")

  def get_M(self):
    ''' returns numpy array '''
    return self.M

  def get_dim(self)-> int:
    return self.dim

  def S(self)->float:
    ''' Finds the von-Neumann entropy of a quantum state. '''

    #print('Calculating Entropy')

    eig=tf.cast(self.eig(),tf.float64)
    ent=tf.keras.losses.CategoricalCrossentropy()

    s=ent(eig,eig)/tf.cast(tf.math.log(2.),dtype=tf.float64)

    #leig=tf.where(eig**2<1e-8,tf.ones(tf.shape(eig)),eig)
    #s=-tf.math.reduce_sum(eig* tf.math.log(leig))/tf.math.log(2.)

    return s



Quantum Channel Class

In [ ]:
class Q_Channel():

  def __init__(self,p : typing.Union[np.ndarray,list,tuple],Ak : typing.Callable[...,typing.List[np.ndarray]]):
    '''
        p contains the parameters in the channel
        Ak are functions that take in p as arguments and return a List of
        Kraus operators
    '''

    self.p=np.array(p)
    self.Ak=Ak

    #self.test()

  def C(self,r: DO)-> DO :
    ''' perform the action of the quantum operation on the quantm state'''
    A_=self.Ak(self.p)

    M=tf.zeros([r.get_dim(),r.get_dim()],dtype=tf.complex128)

    for A_k in A_:
      M+=A_k@ (r.get_M()@ tf.math.conj(tf.transpose(A_k)))

    R=DO(r.get_dim(),M)

    return R

  def test(self):
    ''' Checks if the Kraus operators are valid, for the given set of
    parameters'''

    A_=self.Ak(self.p)

    M=tf.zeros(tf.shape(A_[-1]),dtype=tf.complex128)
    for A_k in A_:
      #M+=A_k@ (r.get_M()@ tf.math.conj(tf.transpose(A_k)))
      M+=tf.transpose(A_k,conjugate=True)@A_k

    if np.allclose(M.numpy(),np.identity(np.size(M,axis=-1))):
      #print("Kraus operators sums to 1")
      None
    else:
      print(M.numpy())
      raise Exception("Invalid set of Kraus operators")

  def disp_Kraus(self):
    ''' Prints the Kraus operators of the quantum channel'''
    print(' ');print(' ')
    print('=========================')
    print('   Channel Details')
    print('   ***************')
    print('')
    print("parameter list:",end=' ')
    print(self.p)
    A_=self.Ak(self.p)

    print('Kraus operators:')
    print('----------------------')
    for A_k in A_:
      print(A_k.numpy())
      print('----------------------')
    print('=========================')

  def return_kraus(self):
    return self.Ak(self.p)

Defining Quantum Channels

In [ ]:
def PauliChannel_Kraus(p: float)-> typing.List[np.ndarray]:
  A1=tf.convert_to_tensor(np.sqrt(1-p[0]-p[1]-p[2])*np.identity(2),dtype=tf.complex128)
  A2=tf.convert_to_tensor(np.sqrt(p[0])*np.array([[0,1],[1,0]]),dtype=tf.complex128)
  A3=tf.convert_to_tensor(np.sqrt(p[1])*np.array([[0,-1j],[1j,0]]),dtype=tf.complex128)
  A4=tf.convert_to_tensor(np.sqrt(p[2])*np.array([[1,0],[0,-1]]),dtype=tf.complex128)

  return [A1,A2,A3,A4]


De Phase Channel, Pauli channel is already defined

In [ ]:
def Q_DePhaseKraus(p: float):
  A1=tf.cast(tf.math.sqrt(1-p),tf.complex128)*tf.eye(2,dtype=tf.complex128)
  A2=tf.cast(tf.math.sqrt(p),tf.complex128)*tf.constant([[1.,0.],[0.,-1.]],dtype=tf.complex128)

  return [A1,A2]


De Polarising Channel

In [ ]:
def Q_ErKraus(p: float)-> typing.List[np.ndarray]:
  A1=tf.convert_to_tensor(np.sqrt(1-3*p/4)*np.identity(2),dtype=tf.complex128)
  A2=tf.convert_to_tensor(np.sqrt(p)*np.array([[0,1],[1,0]])/2,dtype=tf.complex128)
  A3=tf.convert_to_tensor(np.sqrt(p)*np.array([[0,-1j],[1j,0]])/2,dtype=tf.complex128)
  A4=tf.convert_to_tensor(np.sqrt(p)*np.array([[1,0],[0,-1]])/2,dtype=tf.complex128)

  return [A1,A2,A3,A4]


Finding Partial Trace

In [ ]:
def bi_PTr(AB : DO, A_dim : int, B_dim : int, TraceB=True) -> DO:


  if TraceB:
    M=tf.reshape(AB.M,[A_dim,B_dim,A_dim,B_dim])
    M_=tf.transpose(M,perm=[0,2,1,3])
    #sysM= np.trace(M,axis1=1,axis2=3)
    sysM= tf.linalg.trace(M_)
    PT=DO(A_dim,sysM)

  else:
    M=tf.reshape(AB.M,[A_dim,B_dim,A_dim,B_dim])
    M_=tf.transpose(M,perm=[1,3,0,2])
    #sysM= np.trace(M,axis1=0,axis2=2)
    sysM= tf.linalg.trace(M_)
    PT=DO(B_dim,sysM)

  return PT

Finding Coherent Information, but not to be used in relation to optimization

In [ ]:
def I(qr : DO,sys_dim : int, env_dim : int, N : Q_Channel, QR : bool=True):
  ''' qr is the initial purified state. It is also assumed that
      kronecker product is taken as kron(Q,R). If this is not true,
      pass QR=False

      Also note that N only acts on system space.'''

  # Create the reduced DO.

  if QR:
    q=bi_PTr(qr,sys_dim,env_dim)   # Redundant; Is it? qr always changes right?
  else:
    q=bi_PTr(qr,env_dim,sys_dim,TraceB=False)

  # Create the I \outer N. This is also depends on QR value

  #RI=np.identity(env_dim)
  RI=tf.eye(env_dim)

  A=N.return_kraus()

  if QR:
    AI=[np.kron(A_k,RI) for A_k in A]

  else:
    AI=[np.kron(RI,A_k) for A_k in A]

  IN=Q_Channel(N.p,lambda p: AI)  # Redundant

  QR_out=IN.C(qr)
  Q_out=N.C(q)

  CI=Q_out.S()-QR_out.S()

  return (CI,IN,QR_out,Q_out)

Coherent information part but in section to use for the optimization part

RDO Generator

In [ ]:
def RDO_Gen(qr : DO,sys_dim : int, env_dim : int, QR : bool=True):
  if QR:
    q=bi_PTr(qr,sys_dim,env_dim)
  else:
    q=bi_PTr(qr,env_dim,sys_dim,TraceB=False)

  return q

QR Channel Builder

In [ ]:
def QR_Channel_Builder(N : Q_Channel,env_dim,QR : bool=True):

  RI=np.identity(env_dim)
  A=N.return_kraus()
  if QR:
    AI=[ tf.convert_to_tensor(np.kron(A_k.numpy(),RI),dtype=tf.complex128) for A_k in A]

  else:
    AI=[ tf.convert_to_tensor(np.kron(RI,A_k.numpy()),dtype=tf.complex128) for A_k in A]

  IN=Q_Channel(N.p,lambda p: AI)

  return IN

N-Times Channel Builder

In [ ]:
def NT_Channel_Builder(N: Q_Channel,n : int):
  ''' Builds a quantum channel corresponding to N channel uses.'''

  A=N.return_kraus()

  NA=A

  for _ in range(n-1):
    NA_temp=[]

    for NA_k in NA:
      for A_k in A:

        NA_temp.append(tf.convert_to_tensor(\
           np.kron(NA_k.numpy(),A_k.numpy()),dtype=tf.complex128))

    NA=NA_temp

  NA_C=Q_Channel(N.p,lambda p: NA)

  return NA_C



NN States

NN State builder

In [ ]:
def NN_builder(qubits,width,hiddenLayers,Activation,debug=None):  # how are the weights incorporated
  '''
      debug in ['Summary']
  '''
  NN_State=tf.keras.Sequential()

  def a_cos(x):
    return K.cos(x)

  NN_State.add(tf.keras.layers.InputLayer(input_shape=(qubits,)))
  NN_State.add(tf.keras.layers.Dense(width,activation=a_cos)) # As in the paper

  for _ in range(1,hiddenLayers):
    NN_State.add(tf.keras.layers.Dense(width,activation=Activation))
  NN_State.add(tf.keras.layers.Dense(2,activation='tanh'))

  if debug in ['Summary']:
   NN_State.summary()

  return NN_State

In [ ]:
def return_INPUT(qubits):
  In=[]
  for _ in range(2**qubits):
    #input=tf.Variable(np.expand_dims(np.array(list(np.binary_repr(_).zfill(qubits))).astype(np.float64),axis=0))
    input=tf.Variable(np.array(list(np.binary_repr(_).zfill(qubits))).astype(np.float64))
    #print(input)
    input=input-0.5*np.ones(np.shape(input))
    In.append(input)
  In=tf.convert_to_tensor(In)

  return In

NN_2_DO

In [ ]:
def NN_DO(NN,qubits,Input,debug=None):

  '''
        debug in ['Out','All','Wf','Wf-Do','Do']
  '''
  #In=[]

  #for _ in range(2**qubits):
    #input=tf.Variable(np.expand_dims(np.array(list(np.binary_repr(_).zfill(qubits))).astype(np.float64),axis=0))
    #print(input)
    #input=input-0.5*np.ones(np.shape(input))
    #In.append(input)


  #In=tf.convert_to_tensor(In)

  Out=NN(Input)


  if debug in ['Out','All']:
    print('**********************')
    print('Output:')
    print(Out.numpy())
    print('**********************')

  wf=tf.zeros(shape=(2**qubits,1),dtype=tf.complex128)

  wf=tf.cast(Out[:,0],tf.complex128)+tf.cast(tf.complex(0.,1.),tf.complex128)*tf.cast(Out[:,1],tf.complex128)

  wf=tf.reshape(wf,shape=(-1,1))


  def norm(res):
   return tf.math.sqrt(tf.reduce_sum(res*tf.math.conj(res)))

  wf=wf/norm(wf)


  if debug in ['Wf','All','Do-Wf']:
   print('**********************')
   print('Wavefunction:')
   print(wf.numpy())
   print('**********************')

  do=wf@tf.transpose(wf,conjugate=True)



  if debug in ['Do','Do-Wf','All']:
    print('**********************')
    print('DO:')
    print(do.numpy())
    print('**********************')

  NN_State=DO(2**qubits,do)

  #CI,InN,QR_out,Q_out= I(NN_State,2,2,Q_DePhase_Ch)
  #print('CI:',end=' ');print(CI)

  return NN_State

DEAP-EA

In [ ]:
from deap import creator
from deap import tools
from deap import base
from scoop import futures
# Create things
creator.create("FitnessMin",base.Fitness,weights=(-1.0,))
creator.create("Individual",list,fitness=creator.FitnessMin)

#ToolBox
toolbox=base.Toolbox()

def gen_Individual(ind_class,SHAPES):

  W=[]
  for shape in SHAPES:
    weight=np.random.uniform(low=-1.,high=1.,size=shape)
    W.append(weight)

  ind=ind_class(W)

  return ind

'''
toolbox.register("individual",gen_Individual,creator.Individual)
toolbox.register("population",tools.initRepeat,list,toolbox.individual)
'''


'\ntoolbox.register("individual",gen_Individual,creator.Individual)\ntoolbox.register("population",tools.initRepeat,list,toolbox.individual)\n'

In [ ]:
class FitnessClass():
  def __init__(self,NN_State,qubits,N,IN,sys_qubits,env_qubits,reg,QR=True):
    self.NN_State=NN_State
    self.qubits=qubits
    self.N=N
    self.IN=IN
    self.sys_qubits=sys_qubits
    self.env_qubits=env_qubits
    self.reg=reg
    self.QR=QR

  def Fitness(self,W):

    In=return_INPUT(self.qubits)

    self.NN_State.set_weights(W)

    sys_dim=2**self.sys_qubits
    env_dim=2**self.env_qubits

    DOO=NN_DO(self.NN_State,self.qubits,In,debug=None)
    rDO=RDO_Gen(DOO,sys_dim,env_dim,self.QR)


    QR_out=self.IN.C(DOO)
    Q_out=self.N.C(rDO)

    CI=Q_out.S()-QR_out.S()

    loss= -CI + self.reg(rDO)
    return (loss.numpy(),)

In [ ]:
def gen_fitness(NN_State,qubits,N,IN,sys_qubits,env_qubits,QR,reg):

  In=[]
  In=return_INPUT(qubits)
  In=tf.convert_to_tensor(In)

  def fitness(W):

    if type(W[0])==type((1,2)):
      print(W[0])
    NN_State.set_weights(W)

    sys_dim=2**sys_qubits
    env_dim=2**env_qubits

    DOO=NN_DO(NN_State,qubits,In,debug=None)
    rDO=RDO_Gen(DOO,sys_dim,env_dim,QR)


    QR_out=IN.C(DOO)
    Q_out=N.C(rDO)

    CI=Q_out.S()-QR_out.S()

    loss= -CI + reg(rDO)
    return (loss.numpy(),)

  return fitness

Crossover function

In [ ]:
def cx2point(ind1,ind2):
  assert len(ind1)==len(ind2), "Assert error: cx2point: 1"

  IND1=[];IND2=[]

  for _ in range(len(ind1)):
    i1=np.copy(ind1[_])
    i2=np.copy(ind2[_])

    if ind1[_].ndim==2:
      dim=np.shape(ind1[_])

      r1=np.random.randint(0,dim[0])
      r2=np.random.randint(r1,dim[0])
      r3=np.random.randint(0,dim[1])
      r4=np.random.randint(r3,dim[1])

      i1[r1:r2,r3:r4]=np.copy(ind2[_][r1:r2,r3:r4])
      i2[r1:r2,r3:r4]=np.copy(ind1[_][r1:r2,r3:r4])


    else:
      dim=np.shape(ind1[_])
      r1=np.random.randint(0,dim[0])
      r2=np.random.randint(r1,dim[0])

      i1[r1:r2]=np.copy(ind2[_][r1:r2])
      i2[r1:r2]=np.copy(ind1[_][r1:r2])

    IND1.append(i1)
    IND2.append(i2)

  IND1=creator.Individual(IND1)
  IND2=creator.Individual(IND2)
  return IND1,IND2

Mutation

In [ ]:
def mutGaussian(ind,mu, sigma, indpb):

  IND=[]

  for _ in range(len(ind)):
    i1=np.copy(ind[_])
    i1=tools.mutGaussian(i1,mu,sigma,indpb)[0]

    IND.append(i1)
  IND=creator.Individual(IND)
  return IND,

Get shapes

In [ ]:
def get_Shapes(NN):
  SHAPES=[]
  weights=NN.get_weights()

  for _ in weights:
    SHAPES.append(np.shape(_))

  return SHAPES

Regularisation

In [ ]:
Reg=100

def reg(rDO):
  # Should be large wen rDO is pure

  # When pure, result= Reg
  # When fully mixed,

  trace=tf.cast(tf.linalg.trace(rDO.M @ rDO.M),tf.float64)

  return 0#(Reg*(trace-0.5)**2)

In [ ]:
from scoop import futures
def Channel_action_single(channel,param):
    ######################### Problem Parameters ###################################
    qubits=2
    env_qubits=1
    sys_qubits=1

    dim=2**qubits
    sys_dim=2**sys_qubits
    env_dim=2**env_qubits


    Channel=Q_Channel(param,channel)

    ######################## DEAP PARAMETERS #######################################
    # GA

    ngen=10
    pop_size=100

    # GA
    mu=0.5
    sigma=0.25
    indpb=0.5

    tournsize=3

    ########################## Regularisation #######################################
    Reg=0

    def reg(rDO):
      # Should be large wen rDO is pure

      # When pure, result= Reg
      # When fully mixed,

      trace=tf.cast(tf.linalg.trace(rDO.M @ rDO.M),tf.float64)

      return (Reg*(trace-0.5)**2)

    ################################# DEFINITIONS ##################################
    # Define NN
    NN_State=NN_builder(qubits=qubits,width=10,hiddenLayers=10,Activation='tanh')
    #NN_State.set_weights(wta)#result[0][3])

    # Define Channels
    N=Channel
    IN=QR_Channel_Builder(N,env_dim)

    # GA Operators
    fitness=gen_fitness(NN_State,qubits,N,IN,sys_qubits,env_qubits,QR=True,reg=reg)

    SHAPES=get_Shapes(NN_State)

    toolbox.register("individual",gen_Individual,creator.Individual,SHAPES=get_Shapes(NN_State))
    toolbox.register("population",tools.initRepeat,list,toolbox.individual)

    toolbox.register("evaluate", fitness)
    toolbox.register("mate", cx2point)
    toolbox.register("mutate", mutGaussian, mu=0.5,sigma=0.25,indpb=0.5)
    toolbox.register("select", tools.selTournament, tournsize=tournsize)

    toolbox.register("map",futures.map)
    ################################### MAIN #######################################

    def main():

      pop = toolbox.population(n=500)

      def similar(a,b):
        sim=True
        for _ in range(len(a)):

          if ~np.all(a[_]==b[_]):
            sim=False
        return sim

      hof = tools.HallOfFame(1,similar=similar)
      stats = tools.Statistics(lambda ind: ind.fitness.values)
      stats.register("avg", np.mean)
      stats.register("std", np.std)
      stats.register("min", np.min)
      stats.register("max", np.max)

      pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.5, ngen=30,
                                    stats=stats, halloffame=hof, verbose=True)
      best=hof.items[0]
      print(best.fitness.values[0])

      return best

    if __name__ == "__main__":
      wt=main()

    return wt,wt.fitness.values[0]

In [ ]:
def Channel_action_multi(qubits,sysq,envq,channel,param):
    ######################### Problem Parameters ###################################

    dim=2**qubits
    sys_dim=2**sysq
    env_dim=2**envq


    Channel=Q_Channel(param,channel)
    ChannelM=NT_Channel_Builder(Channel,sysq)

    ######################## DEAP PARAMETERS #######################################
    # GA

    ngen=10
    pop_size=100

    # GA
    mu=0.5
    sigma=0.25
    indpb=0.5

    tournsize=3

    ########################## Regularisation #######################################
    Reg=0

    def reg(rDO):
      # Should be large wen rDO is pure

      # When pure, result= Reg
      # When fully mixed,

      trace=tf.cast(tf.linalg.trace(rDO.M @ rDO.M),tf.float64)

      return (Reg*(trace-0.5)**2)

    ################################# DEFINITIONS ##################################
    # Define NN
    NN_State=NN_builder(qubits=qubits,width=10,hiddenLayers=10,Activation='tanh')
    #NN_State.set_weights(wtb)#result[0][4])

    # Define Channels
    N=ChannelM
    IN=QR_Channel_Builder(N,env_dim)

    # GA Operators
    fitness=gen_fitness(NN_State,qubits,N,IN,sysq,envq,QR=True,reg=reg)

    SHAPES=get_Shapes(NN_State)

    toolbox.register("individual",gen_Individual,creator.Individual,SHAPES=get_Shapes(NN_State))
    toolbox.register("population",tools.initRepeat,list,toolbox.individual)

    toolbox.register("evaluate", fitness)
    toolbox.register("mate", cx2point)
    toolbox.register("mutate", mutGaussian, mu=0.5,sigma=0.25,indpb=0.5)
    toolbox.register("select", tools.selTournament, tournsize=tournsize)

    toolbox.register("map",futures.map)
    ################################### MAIN #######################################

    def main():

      pop = toolbox.population(n=500)

      def similar(a,b):
        sim=True
        for _ in range(len(a)):

          if ~np.all(a[_]==b[_]):
            sim=False
        return sim

      hof = tools.HallOfFame(1,similar=similar)
      stats = tools.Statistics(lambda ind: ind.fitness.values)
      stats.register("avg", np.mean)
      stats.register("std", np.std)
      stats.register("min", np.min)
      stats.register("max", np.max)

      pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.5, ngen=30,
                                    stats=stats, halloffame=hof, verbose=True)
      best=hof.items[0]
      print(best.fitness.values[0])

      return best

    if __name__ == "__main__":
      wt=main()

    return wt,wt.fitness.values[0]

In [ ]:
file=open('Best.txt','a')
file.write('p1    p2    p3  Best CI\n')

def superadditivityfn(para):
  wt1,ci1=Channel_action_single(PauliChannel_Kraus,para)
  wt2,ci2=Channel_action_multi(4,2,2,PauliChannel_Kraus,para)
  return ci2*0.5-ci1,ci1,ci2,wt1,wt2

if __name__ == "__main__":
    with concurrent.futures.ProcessPoolExecutor(8) as executor:
        start_time = time.perf_counter()

        para = [[0.225688,0.00801196,0.0263041],[0.225,0.008,0.027],[0.225,0.009,0.026],[0.226,0.008,0.026],[0.225,0.009,0.027],[0.226,0.009,0.026],[0.226,0.008,0.027],[0.226,0.009,0.027]]
        result = list(executor.map(superadditivityfn, para))
        str="{p0:.2f} {p1:.2f} {p2:.2f} {ci1:=.4f} {ci2:=.4f}\n"
        print(result[0][0])

        for i in np.arange(0,len(para)-1,1):
          if -1*(result[i][0])>0:
            file.write(str.format(p0=para[i][0],p1=para[i][1],p2=para[i][2],ci1=-result[i][1],ci2=-result[i][2]))

#        if -1*(result[0][0])>0:
#          file.write(str.format(p0=para[0][0],p1=para[0][1],p2=para[2],ci1=-result[0][1],ci2=-result[0][2]))

        finish_time = time.perf_counter()
    print(f"Program finished in {finish_time-start_time} seconds")

file.close()

/usr/local/lib/python3.7/dist-packages/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  RuntimeWarning
/usr/local/lib/python3.7/dist-packages/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  RuntimeWarning
/usr/local/lib/python3.7/dist-packages/scoop/fallbacks.py:46: RuntimeWarning: SCOOP was not started properly.
Be sure to start your program with the '-m scoop' parameter. You can find further information in the documentation.
Your map call has been replaced by the builtin serial Python map().
  RuntimeWarning
/usr/local/lib/python3.7/dist-packages/scoop/fallbacks.py:46: Runti

gen	nevals	avg       	std       	min        	max      
0  	500   	0.00952811	0.00932897	2.14978e-08	0.0316732
gen	nevals	avg       	std      	min         	max      
0  	500   	0.00894548	0.0087954	-0.000993802	0.0299516
gen	nevals	avg      	std      	min        	max      
0  	500   	0.0102955	0.0101291	2.14978e-08	0.0339486
gen	nevals	avg       	std       	min        	max      
0  	500   	0.00971454	0.00950957	2.14978e-08	0.0322204
gen	nevals	avg      	std       	min        	max      
0  	500   	0.0090754	0.00891021	4.29957e-08	0.0303421gen	nevals	avg       	std       	min        	max      
0  	500   	0.00934338	0.00915856	2.14978e-08	0.0311478

gen	nevals	avg      	std       	min        	max     
0  	500   	0.0101092	0.00992803	2.14978e-08	0.033402
gen	nevals	avg       	std       	min        	max     
0  	500   	0.00952982	0.00933146	2.14978e-08	0.031695
1  	366   	0.00400061	0.00730557	0          	0.0315151
1  	367   	0.00379543	0.00686999	0          	0.0308229
1  	374   	0.00407675	

KeyboardInterrupt: ignored

In [ ]:
for l in np.arange(0,7,1):
  print(result[l][0],'\n')

NameError: ignored